In [32]:
import numpy as np
import pandas as pd

from scipy.stats import loguniform
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_classification 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold, cross_val_score
from tpot import TPOTClassifier, TPOTRegressor
from deap.gp import Primitive
from functools import partial
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK


1. С помощью make_classification из sklearn.datasets сгенерируйте датасет с 1 000 объектов, 10 признаками и бинарным таргетом.

In [3]:
X, y = make_classification(n_samples=1000, n_features=10, n_classes=2)
X.shape, y.shape

((1000, 10), (1000,))

3. Зададим сетку гиперпараметров и инициализируем модель в виде пайплайна. Параметр penalty будем выбирать равновероятно из  ['l1', 'l2'], а параметр регуляризации С из лог-равномерного распределения может принимать значения [10-4, 102]. 

In [5]:
search_space = {
                'lr__penalty' : ['l1', 'l2'],
                'lr__C' : loguniform.rvs(10**(-4),10**2, size=100)
                }

model = Pipeline([('lr', LogisticRegression(random_state=42, 
                            solver='liblinear'))])

GRID SEARCH

Перебор по сетке будем выполнять с помощью GridSearchCV из sklearn.model_selection. 

4. При его инициализации укажем несколько параметров: 

Модель (пайплайн)
Сетку с параметрами
Количество разбиений для (Stratified)KFold, который используется по умолчанию = 3
Метрику для оценки производительности модели с перекрёстной проверкой на тестовом наборе scoring='accuracy'.
5. Обучите GridSearchCV с параметрами (model_grid = grid_search.fit(X, y)).

Теперь можем посмотреть на лучший score и наилучшие гиперпараметры:

In [6]:
reg_grid = GridSearchCV(model, param_grid=search_space, cv=3, scoring='accuracy')


model_grid = reg_grid.fit(X, y)

In [7]:
model_grid.best_score_

0.927022831214448

In [11]:
model_grid.best_params_

{'lr__C': 0.2010496174027811, 'lr__penalty': 'l2'}

Random Search

Для случайного поиска воспользуемся RandomizedSearchCV из sklearn.model_selection. Помимо модели, параметров, скоринга и cv, зададим n_iter. Он отвечает за количество выбранных комбинаций параметров. Чем больше n_iter, тем дольше будет работать поиск. Соответственно, максимально возможный n_iter приближает RandomizedSearchCV к GridSearchCV. 

6. Задайте n_iter = 70 и инициализируйте RandomizedSearchCV.

7. Обучите полученный оптимизатор на  и  и оцените .best_score_ и .best_params_.

In [9]:
n_iter = 70

reg_rand = RandomizedSearchCV(model, param_distributions=search_space,\
    n_iter=n_iter, cv=3, scoring='accuracy')

model_rand = reg_rand.fit(X, y)

In [10]:
reg_rand.best_score_

0.927019834205463

In [12]:
reg_rand.best_params_

{'lr__penalty': 'l1', 'lr__C': 1.0968082601439908}

TPOT

Для генетического алгоритма воспользуемся библиотекой TPOT.

8. Для установки выполните pip install tpot, после чего импортируйте из tpot TPOTClassifier.

9. Задайте параметры немного иначе, при этом оставив те же значения:

search_space = {
                'penalty' : ['l1', 'l2'],
                'C' : loguniform.rvs(10**(-4),10**2, size=100)
                }
10. При инициализации классификатора добавьте следующие параметры:

generations = 5, (Количество поколений в процессе оптимизации)
population_size = 50, (Число особей, сохраняемых в популяции генетического программирования в каждом поколении)
offspring_size = 25, (Количество потомства, которое нужно произвести в каждом поколении генетического программирования)
verbosity = 2, 
config_dict = {'sklearn.linear_model.LogisticRegression': search_space}, (словарь с гиперпараметрами для оптимизации для выбранной модели)
cv = 3, 
scoring = 'accuracy')
11. Обучите инициализированный классификатор tpot_classifier.fit(X, y).

12. Для извлечения наилучших параметров воспользуйтесь вспомогательным кодом:

args = {}
for arg in tpot_classifier._optimized_pipeline:
    if type(arg) != Primitive:
        try:
            if arg.value.split('__')[1].split('=')[0] in ['C', 'penalty']:
                args[arg.value.split('__')[1].split('=')[0]] = (arg.value.split('__')[1].split('=')[1])
            else:
                args[arg.value.split('__')[1].split('=')[0]] = float(arg.value.split('__')[1].split('=')[1])
        except:
            pass
params = args

In [22]:
search_space = {
                'penalty' : ['l1', 'l2'],
                'C' : loguniform.rvs(10**(-4),10**2, size=100)
                }

tpot_classifier = TPOTClassifier(generations = 5,
                                 population_size = 50, 
                                #  offspring_size = 25,
                                #  verbosity = 2, 
                                 config_dict = {'sklearn.linear_model.LogisticRegression': search_space}, 
                                 cv = 3, scoring = 'accuracy')

tpot_classifier.fit(X, y)

TypeError: TPOTEstimator.__init__() got an unexpected keyword argument 'config_dict'

In [23]:
args = {}
for arg in tpot_classifier._optimized_pipeline:
    if type(arg) != Primitive:
        try:
            if arg.value.split('__')[1].split('=')[0] in ['C', 'penalty']:
                args[arg.value.split('__')[1].split('=')[0]] = (arg.value.split('__')[1].split('=')[1])
            else:
                args[arg.value.split('__')[1].split('=')[0]] = float(arg.value.split('__')[1].split('=')[1])
        except:
            pass
params = args

AttributeError: 'TPOTClassifier' object has no attribute '_optimized_pipeline'

Байесовская оптимизация

Здесь значения гиперпараметров в текущей итерации выбираются с учётом результатов на предыдущем шаге. 

Основная идея алгоритма заключается в следующем: на каждой итерации подбора находится компромисс между исследованием регионов с самыми удачными из найденных комбинаций гиперпараметров и исследованием регионов с большой неопределённостью (где могут находиться ещё более удачные комбинации). Это позволяет во многих случаях найти лучшие значения параметров модели за меньшее количество времени.

Мы рассмотрим библиотеку Hyperopt для подбора гиперпарметров. В ней реализовано три алгоритма оптимизации: 

классический Random Search;
метод байесовской оптимизации Tree of Parzen Estimators (TPE);
Simulated Annealing, метод имитации отжига. 
Hyperopt может работать с разными типами гиперпараметров — непрерывными, дискретными, категориальными и так далее, что является важным преимуществом этой библиотеки.

13. Для установки библиотеки выполните pip install hyperopt.

14. Импортируйте вспомогательные функции:

from functools import partial
from sklearn.model_selection import StratifiedKFold
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
15. Укажите объект для сохранения истории поиска (Trials). Это очень удобно, поскольку можно сохранять, прерывать и затем продолжать процесс поиска гиперпараметров. Запустите сам процесс подбора с помощью функции fmin. Укажите в качестве алгоритма поиска tpe.suggest — байесовскую оптимизацию. Для Random Search нужно указать tpe.rand.suggest.

16. Задайте гиперпараметры с использованием распределения из hyperopt:

search_space = {
                'lr__penalty' : hp.choice(label='penalty', 
                          options=['l1', 'l2']),
                'lr__C' : hp.loguniform(label='C', 
                        low=-4*np.log(10), 
                        high=2*np.log(10))
Они будут иметь всё те же значения.

17. Нам понадобится воспользоваться вспомогательной функцией, которую мы будем оптимизировать:

def objective(params, model,  X_train, y_train):
    """
    Кросс-валидация с текущими гиперпараметрами

    :params: гиперпараметры
    :pipeline: модель
    :X_train: матрица признаков
    :y_train: вектор меток объектов
    :return: средняя точность на кросс-валидации
    """ 

    # задаём модели требуемые параметры    
    model.set_params(**params)
    
    # задаём параметры кросс-валидации (стратифицированная 4-фолдовая с перемешиванием)
    skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=1)

    # проводим кросс-валидацию  
    score = cross_val_score(estimator=model, X=X_train, y=y_train, 
                            scoring='accuracy', cv=skf, n_jobs=-1)

    # возвращаем результаты, которые записываются в Trials()
    return   {'loss': -score.mean(), 'params': params, 'status': STATUS_OK}
Теперь можем приступать к оптимизации.

18. Запустите hyperopt.

trials = Trials()
best = fmin( 
          # функция для оптимизации  
            fn=partial(objective, model=model, X_train=X, y_train=y),
          # пространство поиска гиперпараметров  
            space=search_space,
          # алгоритм поиска
            algo=tpe.suggest,
          # число итераций (можно ещё указать  время поиска) 
            max_evals=40,
          # куда сохранять историю поиска
            trials=trials,
          # random state
            rstate=np.random.RandomState(42),
          # progressbar
            show_progressbar=True
        )
19. Посмотрите на параметры, подобранные с помощью байесовской оптимизации, и сравните с предыдущими результатами. Какая модель имеет наилучший score на кросс-валидации?

print(best)

In [27]:
search_space = {
                'lr__penalty' : hp.choice(label='penalty', 
                          options=['l1', 'l2']),
                'lr__C' : hp.loguniform(label='C', 
                        low=-4*np.log(10), 
                        high=2*np.log(10))}

In [28]:
model = Pipeline([('lr', LogisticRegression(random_state=42, solver='liblinear'))])

In [29]:
def objective(params, model,  X_train, y_train):
    """
    Кросс-валидация с текущими гиперпараметрами

    :params: гиперпараметры
    :pipeline: модель
    :X_train: матрица признаков
    :y_train: вектор меток объектов
    :return: средняя точность на кросс-валидации
    """ 

    # задаём модели требуемые параметры    
    model.set_params(**params)
    
    # задаём параметры кросс-валидации (стратифицированная 4-фолдовая с перемешиванием)
    skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=1)

    # проводим кросс-валидацию  
    score = cross_val_score(estimator=model, X=X_train, y=y_train, 
                            scoring='accuracy', cv=skf, n_jobs=-1)

    # возвращаем результаты, которые записываются в Trials()
    return   {'loss': -score.mean(), 'params': params, 'status': STATUS_OK}

In [36]:
trials = Trials()
best = fmin( 
          # функция для оптимизации  
            fn=partial(objective, model=model, X_train=X, y_train=y),
          # пространство поиска гиперпараметров  
            space=search_space,
          # алгоритм поиска
            algo=tpe.suggest,
          # число итераций (можно ещё указать  время поиска) 
            max_evals=40,
          # куда сохранять историю поиска
            trials=trials,
          # random state
            # rstate=np.random.RandomState(42),
          # progressbar
            show_progressbar=True
        )

  0%|          | 0/40 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 40/40 [00:01<00:00, 27.37trial/s, best loss: -0.931]


In [38]:
print(best)

{'C': 2.3764223753600926, 'penalty': 0}
